# CoppeliaSim: PART 2
* Run the scene workshop1.ttt in CoppeliaSim

In [1]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Robot

In [2]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(self.client_id, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

In [3]:
def follow_traj(robot_position, desired_position, vi, vf, tf):
    # Time duration
    # Time interval
    time_steps = linspace(0, tf, 300)

    x1 = robot_position
    x2 = desired_position
    v1 = vi
    v2 = vf
    a0 = x1
    a1 = v1
    a2 = (3*x2 - 3*x1 - 2*v1*tf - v2 * tf) / (tf**2)
    a3 = (2*x1 + (v1 + v2) * tf  - 2 * x2) / (tf**3)

    # a1 = (desired_position - robot_position) / tf
    # a0 = robot_position

    for t in time_steps:
        polynomial = a0 + a1 * t + a2 * t**2 + a3 * t**3
        #derivative = a1 + 2*a2 * t + 3 * a3 * t**2
        point_traj = polynomial
        #vel_traj = derivative
        #print(point_traj)
        #ref_point.set_position(point_traj)

        # Sensing
        robot_position = r.get_position()


        # Trajectory tracker
        

        px, py, pz = polynomial
        r.set_position([px,py,pz])

        time.sleep(tf/300)
    

In [7]:
r = robot(motor_names = [], frame_name='Quadricopter_target')


# Virtual disk
#disk = robot('Disc', client_id=r.client_id)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere1')
follow_traj(robot_position, desired_position,0, 0, .5)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere5')
desired_position = np.asarray([desired_position[0], desired_position[1], (desired_position[2] + .6)])
follow_traj(robot_position, desired_position, 0, 0, .5)

robot_position = r.get_position()
desired_position = np.asarray([-3.7750, 5.1736, 4.0020])
follow_traj(robot_position, desired_position, 0, 0, .5)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere6')
follow_traj(robot_position, desired_position,0, 0, .5)

robot_position = r.get_position()
desired_position = np.asarray([-3.7750, 2.6, 1.9500])
follow_traj(robot_position, desired_position, 0, 0, .5)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere7')
follow_traj(robot_position, desired_position,0, 0, .5)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere10')
follow_traj(robot_position, desired_position,0, 0, .5)

robot_position = r.get_position()
desired_position = np.asarray([6, -4, 4])
follow_traj(robot_position, desired_position,0, 0, .5)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere8')
follow_traj(robot_position, desired_position,0, 0, .5)

robot_position = r.get_position()
desired_position = np.asarray([-1.65, -7.075, 4])
follow_traj(robot_position, desired_position,0, 0, .5)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere9')
follow_traj(robot_position, desired_position,0, 0, .5)

r.close_connection() 

Robot connected
Connection closed


Robot connected
Connection closed
